In [10]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate
import math
import scipy.fft as fft
from functools import partial
from scipy import signal
#%matplotlib inline
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc
import pickle

In [11]:
size = 30
plt.rcParams['figure.figsize'] = [5, 5]

In [12]:
def showGame(result, delay = 45):
    exit = False
    while not exit:
        for i in range(len(result)):
            img = np.zeros([size, size, 3], dtype=float)
            img[:,:,0] = result[i]*0.1
            img[:,:,1] = result[i]*0.4
            img[:,:,2] = result[i]*0.9

            #vis2 = cv2.cvtColor(result[i].astype(np.float32), cv2.COLOR_GRAY2BGR)
            resized = cv2.resize(img, (600, 600), 0, 0, interpolation = cv2.INTER_NEAREST)
            cv2.imshow("game", resized)
            key = cv2.waitKey(delay) 
            if key!=-1:
                exit = True
                break

    cv2.destroyAllWindows() 

In [13]:
def saveState(data, name, epoch):
    with open('gl/'+str(name)+"_"+str(epoch)+".gl", 'wb') as f:
        pickle.dump(data, f)
        
def readState(name, epoch):
    with open('gl/'+str(name)+"_"+str(epoch)+".gl", 'rb') as f:
        return pickle.load(f)

In [14]:
kernel = np.ones((3, 3), dtype=int)
kernel[1, 1] = 0

def computeStep(inp):
    def cellLife(n, cell):
        if cell == 1:
            if n <=1:
                return 0
            if n>=4:
                return 0
            if (n==2 or n==3):
                return 1
        else:
            if n == 3:
                return 1
            else:
                return 0
            
    def cellLifeRow(n, cell):
        return list(map(cellLife, n, cell))
    
    counter = signal.convolve(inp, kernel, mode='same')
    n_step = map(cellLifeRow, counter, inp)
    #print(np.array(list(n_step)))

    return np.array(list(n_step))

def computeIterations(inp, iters):
    s = inp.shape
    res = np.zeros(s[0]*s[1]*iters, dtype = int).reshape(iters, s[0], s[1])
    res[0] = np.copy(inp)
    
    for i in range(1, iters):
        res[i] = computeStep(res[i-1])
        
    return res

In [24]:
start = np.zeros(size*size).reshape(size,size)
start = np.random.choice([0, 1], p=[0.8, 0.2], size=(size, size))
#np.around(np.random.rand(size, size))

res = computeIterations(start, 20)

In [27]:
showGame(res, 80)

In [26]:
for i in range(len(res)):
    saveState(res[i], "2", i)

In [9]:
#dt = readState("0", 1)
#showGame([dt], 80)